In [71]:
from scipy.sparse import csr_matrix, hstack

from sklearn.linear_model import Ridge
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split, cross_val_score
import lightgbm as lgb
import pandas as pd
import jieba

In [72]:
train = pd.read_csv("./input/train_first.csv")
test = pd.read_csv("./input/predict_first.csv")

In [73]:
data = pd.concat([train, test])

In [74]:
data

,Discuss,Id,Score
0,好大的一个游乐公园，已经去了2次，但感觉还没有玩够似的！会有第三，第四次的,201e8bf2-77a2-3a98-9fcf-4ce03914e712,5.0
1,新中国成立也是在这举行，对我们中国人来说有些重要及深刻的意义！,f4d51947-eac4-3005-9d3c-2f32d6068a2d,4.0
2,庐山瀑布非常有名，也有非常多个瀑布，只是最好看的非三叠泉莫属，推荐一去,74aa7ae4-03a4-394c-bee0-5702d3a3082a,4.0
3,个人觉得颐和园是北京最值的一起的地方，不过相比下门票也是最贵的，比起故宫的雄伟与气势磅礴，颐...,099661c2-4360-3c49-a2fe-8c783764f7db,5.0
4,迪斯尼一日游,97ca672d-e558-3542-ba7b-ee719bba1bab,5.0
5,方便,3b7f3f2e-886f-3a68-a810-2c37cfd728d3,4.0
6,看水看山都可以。感受古人的智慧结晶，秋景美丽如画，红黄绿相间！对于身体状况不佳的人来说，走平...,88914409-bd13-3d47-b5a2-691177dde8fd,4.0
7,赞,bf13ec92-6079-3451-ade3-88020cb0dcb5,5.0
8,唯一糟点,489c3d94-9c44-3cf2-949c-1b507c374c69,5.0
9,周未周边游,285bba78-16a3-3c1d-b648-baa483883ee3,5.0


In [75]:
data['comments'] = data['Discuss'].apply(lambda x : ' '.join(jieba.cut(x)))

In [76]:
data

,Discuss,Id,Score,comments
0,好大的一个游乐公园，已经去了2次，但感觉还没有玩够似的！会有第三，第四次的,201e8bf2-77a2-3a98-9fcf-4ce03914e712,5.0,好大 的 一个 游乐 公园 ， 已经 去 了 2 次 ， 但 感觉 还 没有 玩够 似的 ！...
1,新中国成立也是在这举行，对我们中国人来说有些重要及深刻的意义！,f4d51947-eac4-3005-9d3c-2f32d6068a2d,4.0,新 中国 成立 也 是 在 这 举行 ， 对 我们 中国 人 来说 有些 重要 及 深刻 的...
2,庐山瀑布非常有名，也有非常多个瀑布，只是最好看的非三叠泉莫属，推荐一去,74aa7ae4-03a4-394c-bee0-5702d3a3082a,4.0,庐山 瀑布 非常 有名 ， 也 有 非常 多个 瀑布 ， 只是 最 好看 的 非 三叠 泉莫...
3,个人觉得颐和园是北京最值的一起的地方，不过相比下门票也是最贵的，比起故宫的雄伟与气势磅礴，颐...,099661c2-4360-3c49-a2fe-8c783764f7db,5.0,个人 觉得 颐和园 是 北京 最值 的 一起 的 地方 ， 不过 相比 下 门票 也 是 最...
4,迪斯尼一日游,97ca672d-e558-3542-ba7b-ee719bba1bab,5.0,迪斯尼 一日游
5,方便,3b7f3f2e-886f-3a68-a810-2c37cfd728d3,4.0,方便
6,看水看山都可以。感受古人的智慧结晶，秋景美丽如画，红黄绿相间！对于身体状况不佳的人来说，走平...,88914409-bd13-3d47-b5a2-691177dde8fd,4.0,看水 看山 都 可以 。 感受 古人 的 智慧结晶 ， 秋景 美丽 如画 ， 红黄绿 相间 ...
7,赞,bf13ec92-6079-3451-ade3-88020cb0dcb5,5.0,赞
8,唯一糟点,489c3d94-9c44-3cf2-949c-1b507c374c69,5.0,唯一 糟点
9,周未周边游,285bba78-16a3-3c1d-b648-baa483883ee3,5.0,周未 周边游


In [77]:
data['comments']

0        好大 的 一个 游乐 公园 ， 已经 去 了 2 次 ， 但 感觉 还 没有 玩够 似的 ！...
1        新 中国 成立 也 是 在 这 举行 ， 对 我们 中国 人 来说 有些 重要 及 深刻 的...
2        庐山 瀑布 非常 有名 ， 也 有 非常 多个 瀑布 ， 只是 最 好看 的 非 三叠 泉莫...
3        个人 觉得 颐和园 是 北京 最值 的 一起 的 地方 ， 不过 相比 下 门票 也 是 最...
4                                                  迪斯尼 一日游
5                                                       方便
6        看水 看山 都 可以 。 感受 古人 的 智慧结晶 ， 秋景 美丽 如画 ， 红黄绿 相间 ...
7                                                        赞
8                                                    唯一 糟点
9                                                   周未 周边游
10       景点 服务 不错 ， 就是 排队   太长 了 ， 好玩 的 项目 都 是 人 ， 晚上 的...
11                                               绍兴 护城河 夜游
12                              感觉 还 不错 ， 作为 一日游 不错 的 选择 ～
13                                           有趣 hai   xing
14       荡气回肠 ， 10 年 去 的 ， 居然 没有 留下来 照片 ， 必然 要 再 去 ！ < ...
15       景色 超级 棒 ， 有 美丽 的 故事 ， 可以 乘船 游览 ， 也 可以 沿湖 浏览 ， ...
16       南锣鼓巷 是 北京市 中心 一条 老 胡同 ， 因为 其 地理位置 靠近 什刹海 ， 成为 .

In [78]:
 tv = TfidfVectorizer(max_features=50000)
 X_comments = tv.fit_transform(data.comments)

In [79]:
X_comments

<130000x50000 sparse matrix of type '<class 'numpy.float64'>'
	with 1734791 stored elements in Compressed Sparse Row format>

In [80]:
X_t = X_comments[:train.shape[0]]
X_te = X_comments[train.shape[0]:]

In [81]:
X_te.shape

(30000, 50000)

In [82]:
y = train.Score.values
x_train, x_val, y_train, y_val = train_test_split(X_t, y, test_size = 0.1, random_state=42)

In [83]:
x_val.shape

(10000, 50000)

In [84]:
lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_val, y_val, reference=lgb_train)

In [85]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class' : 6,
    'metric': 'multi_error',
}

In [86]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=3000,
                valid_sets=lgb_eval,
                early_stopping_rounds=50)

[1]	valid_0's multi_error: 0.3949
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's multi_error: 0.3941
[3]	valid_0's multi_error: 0.3937
[4]	valid_0's multi_error: 0.3953
[5]	valid_0's multi_error: 0.3949
[6]	valid_0's multi_error: 0.3945
[7]	valid_0's multi_error: 0.3943
[8]	valid_0's multi_error: 0.3951
[9]	valid_0's multi_error: 0.3944
[10]	valid_0's multi_error: 0.3946
[11]	valid_0's multi_error: 0.3942
[12]	valid_0's multi_error: 0.3936
[13]	valid_0's multi_error: 0.3936
[14]	valid_0's multi_error: 0.3947
[15]	valid_0's multi_error: 0.3942
[16]	valid_0's multi_error: 0.3935
[17]	valid_0's multi_error: 0.3931
[18]	valid_0's multi_error: 0.3923
[19]	valid_0's multi_error: 0.3929
[20]	valid_0's multi_error: 0.392
[21]	valid_0's multi_error: 0.391
[22]	valid_0's multi_error: 0.391
[23]	valid_0's multi_error: 0.3903
[24]	valid_0's multi_error: 0.3905
[25]	valid_0's multi_error: 0.3896
[26]	valid_0's multi_error: 0.3894
[27]	valid_0's multi_error: 0.3891
[28]	

In [87]:
y_val

array([5, 5, 5, ..., 4, 5, 4])

In [88]:
print('Start predicting...')
# predict
y_pred = gbm.predict(X_te, num_iteration=gbm.best_iteration)

Start predicting...


In [89]:
y_pred

array([[  1.02074999e-16,   2.16257277e-03,   3.71120457e-03,
          1.31568605e-02,   1.65827920e-01,   8.15141442e-01],
       [  1.04247014e-16,   1.01018201e-03,   2.99684811e-03,
          1.93025844e-02,   1.66311175e-01,   8.10379211e-01],
       [  1.06017181e-16,   1.17873314e-03,   7.03933393e-03,
          1.03584372e-01,   3.54675617e-01,   5.33521944e-01],
       ..., 
       [  1.87779534e-16,   1.21402048e-03,   2.99462243e-02,
          2.15112306e-01,   2.76485860e-01,   4.77241589e-01],
       [  1.37598314e-16,   7.47458132e-03,   4.32631506e-03,
          1.20482904e-01,   2.17702535e-01,   6.50013665e-01],
       [  1.53357707e-16,   3.31472179e-03,   4.65003534e-03,
          1.60749787e-01,   2.76161699e-01,   5.55123757e-01]])

In [90]:
y_pred = pd.DataFrame(y_pred)


In [91]:
y_1 = y_pred[0]*1 + y_pred[1]*1 + y_pred[2]*2+y_pred[3]*3+y_pred[4]*4 +y_pred[5]*5

In [92]:
y_1[y_1>4.7] = 5

In [93]:
sub = pd.DataFrame()
sub['id'] = pd.DataFrame(test["Id"])
sub['Score'] = pd.DataFrame(y_1)

In [94]:
sub

,id,Score
0,9a1caf96-681e-3c11-b588-43ac742d7fd2,5.000000
1,82b450db-65c2-351c-84fb-761d76582680,5.000000
2,2eec4606-590c-3fa2-b846-7f92441c54a6,4.412323
3,509f9a68-ac41-35ff-9d2e-2fc12f73ed7f,4.536333
4,395f4b22-1c5f-328a-a19d-5065e0530cbc,5.000000
5,6f1202b2-0c1a-3d79-a675-453aad931e3e,4.509231
6,b4bc951f-ce1e-32ae-b27c-bbecba1fd0ee,4.552376
7,30369496-e4b1-30b9-b427-fe23fd1712d6,3.743906
8,704a36e8-47b1-3b93-bb6a-43610c81e1a0,4.515514
9,8a5711e2-b855-3088-adcc-a4d634e2cbb9,5.000000


In [95]:
sub.to_csv('tf_idf.csv',index=False,header=False)